# Library import

In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
# faster rcnn model이 포함된 library
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm

# Dataset 생성

In [2]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)

    def __getitem__(self, index: int):
        
        image_id = self.coco.getImgIds(imgIds=index)

        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        image = torch.tensor(image, dtype=torch.float32).permute(2,0,1)

        return image
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

# inference function

In [3]:
def inference_fn(test_data_loader, model, device):
    outputs = []
    for images in tqdm(test_data_loader):
        # gpu 계산을 위해 image.to(device)
        images = list(image.to(device) for image in images)
        output = model(images)
        for out in output:
            outputs.append({'boxes': out['boxes'].tolist(), 'scores': out['scores'].tolist(), 'labels': out['labels'].tolist()})
    return outputs

# Main

In [4]:
def main():
    annotation = '../dataset/test.json' # annotation 경로
    data_dir = '../dataset' # dataset 경로
    test_dataset = CustomDataset(annotation, data_dir)
    score_threshold = 0.3
    fold = [0,1,2,3,4]
    #check_point = f'/opt/ml/detection/faster_rcnn/baseline_2/checkpoints/fold{fold_ind}_retinanet_resnet50_fpn_4.pth' # 체크포인트 경로
    
    
    test_data_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=4
    )
    for fold_ind in fold:
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        print(device)

        # torchvision model 불러오기
        model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)
        num_classes = 11  # 10 class + background
        # get number of input features for the classifier
        # in_features = model.roi_heads.box_predictor.cls_score.in_features
        # model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        model.to(device)
        model.load_state_dict(torch.load(f'/opt/ml/detection/faster_rcnn/baseline_2/checkpoints/fold{fold_ind}_retinanet_resnet50_fpn_4.pth'))
        model.eval()

        outputs = inference_fn(test_data_loader, model, device)
        prediction_strings = []
        file_names = []
        coco = COCO(annotation)

        # submission 파일 생성
        for i, output in enumerate(outputs):
            prediction_string = ''
            image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
            for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
                if score > score_threshold: 
                    # label[1~10] -> label[0~9]
                    prediction_string += str(label-1) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
                        box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
            prediction_strings.append(prediction_string)
            file_names.append(image_info['file_name'])
        submission = pd.DataFrame()
        submission['PredictionString'] = prediction_strings
        submission['image_id'] = file_names
        submission.to_csv(f'/opt/ml/detection/faster_rcnn/baseline_2/submission_test/fold{fold_ind}_retinanet_resnet50_fpn_test_4_0.3.csv', index=None)
        print(submission.head())

In [5]:
if __name__ == '__main__':
    main()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
cuda


100%|██████████| 609/609 [03:20<00:00,  3.04it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9469152688980103 596.1566772460938 520.197...  test/0000.jpg
1  3 0.6367285251617432 342.8404541015625 257.674...  test/0001.jpg
2  1 0.7887363433837891 308.06097412109375 302.50...  test/0002.jpg
3  9 0.7987083196640015 156.75015258789062 264.82...  test/0003.jpg
4  0 0.5486242175102234 190.6232452392578 252.368...  test/0004.jpg
cuda


100%|██████████| 609/609 [03:17<00:00,  3.09it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9580594897270203 221.02891540527344 57.160...  test/0000.jpg
1  5 0.7363812327384949 345.92254638671875 251.11...  test/0001.jpg
2  1 0.8553988337516785 303.73748779296875 287.67...  test/0002.jpg
3  9 0.8718125224113464 170.0378875732422 268.634...  test/0003.jpg
4  1 0.5077464580535889 195.24609375 312.53820800...  test/0004.jpg
cuda


...

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9792993664741516 221.044921875 52.70486068...  test/0000.jpg
1  5 0.7916504144668579 343.6021728515625 250.964...  test/0001.jpg
2  1 0.8382615447044373 304.1420593261719 276.853...  test/0002.jpg
3  9 0.8115426301956177 152.90928649902344 236.13...  test/0003.jpg
4  0 0.5886224508285522 195.41343688964844 260.08...  test/0004.jpg
cuda


100%|██████████| 609/609 [03:14<00:00,  3.13it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9862715005874634 220.6200408935547 55.3414...  test/0000.jpg
1  5 0.7912735342979431 340.4035339355469 250.758...  test/0001.jpg
2  1 0.8836820125579834 306.0204162597656 266.623...  test/0002.jpg
3  9 0.8820315599441528 159.86944580078125 249.01...  test/0003.jpg
4  0 0.6713767647743225 194.69149780273438 258.19...  test/0004.jpg
cuda


100%|██████████| 609/609 [03:14<00:00,  3.13it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9816142320632935 223.25071716308594 56.211...  test/0000.jpg
1  5 0.8838589191436768 340.8896789550781 248.781...  test/0001.jpg
2  1 0.896791934967041 302.9183044433594 273.0691...  test/0002.jpg
3  9 0.9141399264335632 154.6496124267578 230.399...  test/0003.jpg
4  1 0.5481130480766296 193.02554321289062 294.67...  test/0004.jpg
